## Hyperparameter Tuning and Feature Selection


## 1. Introduction


In this notebook, we tune the hyperparameters of the top-performing model LightGBM using Optuna. Our objective is to improve AUC-ROC scores beyond the default settings by finding the optimal configuration of model parameters.



## Part 1: Hyperparameter Tuning


## 2. Setup and Data Preparation

In this section, we import all required libraries and prepare the dataset for hyperparameter tuning. We use a preprocessed version of the dataset and split it into training and testing sets using stratified sampling to preserve class distribution.


In [ ]:
# Import Libraries and General Setup For LightGBM Tuning
import optuna
import shap
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from lightgbm import early_stopping, log_evaluation, LGBMClassifier

df = pd.read_csv("data/train_preprocessed.csv")

X = df.drop(columns=["loan_status", "id"])  
y = df["loan_status"]

X_train_lgbm, X_test_lgbm, y_train_lgbm, y_test_lgbm = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

## 3. Define Objective Function (Optuna)

We use Optuna to define and optimize our objective function for the LightGBM model.


In [ ]:
# LightGBM Objective Function
def objective_lgbm(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "boosting_type": "gbdt",
        "verbosity": -1,
        "random_state": 42,
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "max_depth": trial.suggest_int("max_depth", 3, 7),
        "num_leaves": trial.suggest_int("num_leaves", 15, 63),
        "subsample": trial.suggest_float("subsample", 0.7, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.7, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 100),
    }

    model = LGBMClassifier(**params)
    model.fit(
    X_train_lgbm, y_train_lgbm,
    eval_set=[(X_test_lgbm, y_test_lgbm)],
    eval_metric="auc",
    callbacks=[
        early_stopping(20, verbose=False),
        log_evaluation(0)  
    ]
)
    preds = model.predict_proba(X_test_lgbm)[:, 1]
    return roc_auc_score(y_test_lgbm, preds)

## 4. Run Optimization


We run the Optuna study for a limited number of trials and extract the best-performing configuration.


In [218]:
# LightGBM Tuning
study_lgbm = optuna.create_study(direction="maximize")
study_lgbm.optimize(objective_lgbm, n_trials=100)

[I 2025-05-17 15:38:03,626] A new study created in memory with name: no-name-7ac0e5d1-4b24-433c-9f57-42d97948cdc8
[I 2025-05-17 15:38:04,754] Trial 0 finished with value: 0.9484406671298025 and parameters: {'n_estimators': 287, 'learning_rate': 0.01598214157606596, 'max_depth': 6, 'num_leaves': 25, 'subsample': 0.8843848878734599, 'colsample_bytree': 0.8711903558755092, 'min_child_samples': 25}. Best is trial 0 with value: 0.9484406671298025.
[I 2025-05-17 15:38:05,877] Trial 1 finished with value: 0.9592515535585554 and parameters: {'n_estimators': 476, 'learning_rate': 0.05584296095691592, 'max_depth': 6, 'num_leaves': 23, 'subsample': 0.9096913966991582, 'colsample_bytree': 0.7066660314251398, 'min_child_samples': 39}. Best is trial 1 with value: 0.9592515535585554.
[I 2025-05-17 15:38:06,533] Trial 2 finished with value: 0.9552263799272912 and parameters: {'n_estimators': 125, 'learning_rate': 0.07261673659954494, 'max_depth': 6, 'num_leaves': 40, 'subsample': 0.8171605616693548, '

## 5. Best Scores and Parameters


After running Optuna, we extract and display the best AUC score of the model.


In [219]:
print("LightGBM Best AUC:", study_lgbm.best_value)


LightGBM Best AUC: 0.9603392082521507


## 6. Retrain and Evaluate


We retrain the model using the best-found parameters and evaluate final AUC-ROC performance on the hold-out test set.


In [220]:
# Re-train Best LightGBM Model On Full Training Data 
best_lgbm_model = LGBMClassifier(
    **study_lgbm.best_params,
    random_state=42
)
best_lgbm_model.fit(X_train_lgbm, y_train_lgbm)
lgbm_probs = best_lgbm_model.predict_proba(X_test_lgbm)[:, 1]
lgbm_auc_final = roc_auc_score(y_test_lgbm, lgbm_probs)

## 7. Summary of Results




In [221]:
print("Final LightGBM AUC on Test Set:", lgbm_auc_final)


Final LightGBM AUC on Test Set: 0.9603392082521507


Tuning improved performance over the baseline model.

## Part 2: SHAP-Based Feature Selection


## 8. SHAP Introduction

In this section, we use SHAP values to interpret and rank feature importance for LightGBM. We then retrain the model using only the most impactful features and evaluate whether this improves AUC-ROC on the test set.




## 9. Compute SHAP Values



In [222]:
# LightGBM SHAP Values
explainer_lgbm = shap.TreeExplainer(best_lgbm_model)
shap_values_lgbm = explainer_lgbm.shap_values(X_train_lgbm)

/Users/nicolasgarzon/Codes/Loan Approval Prediction/.venv/lib/python3.12/site-packages/shap/explainers/_tree.py:583: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(


## 10. Calculate SHAP Importances



In [223]:
# LightGBM SHAP Importance
shap_importance_lgbm = pd.DataFrame({
    "feature": X_train_lgbm.columns,
    "importance": np.abs(shap_values_lgbm).mean(axis=0)
}).sort_values("importance", ascending=False)

## 11. Select Top Features



In [224]:
# LightGBM Top Features
top_features_lgbm = shap_importance_lgbm[shap_importance_lgbm["importance"] > 0.001]["feature"].tolist()
X_train_lgbm_reduced = X_train_lgbm[top_features_lgbm]
X_test_lgbm_reduced = X_test_lgbm[top_features_lgbm]

## 12. Retrain and Evaluate with SHAP Features



In [225]:
# LightGBM Re-Training
model_lgbm_reduced = LGBMClassifier(**study_lgbm.best_params, random_state=42)
model_lgbm_reduced.fit(X_train_lgbm_reduced, y_train_lgbm)

lgbm_probs = model_lgbm_reduced.predict_proba(X_test_lgbm_reduced)[:, 1]
lgbm_auc_shap = roc_auc_score(y_test_lgbm, lgbm_probs)

## 13. Summary of SHAP Results




In [226]:
print("LightGBM AUC after SHAP-based feature selection:", lgbm_auc_shap)


LightGBM AUC after SHAP-based feature selection: 0.9597482636873584


SHAP-based feature selection did not improve performance beyond the tuned model.

## 14. Interpretation and Insights

- **Model selection rationale**: LightGBM outperformed other models across all stages, achieving a best validation AUC of **0.9603** and maintaining that performance on the test set. Even after SHAP-based feature selection, the AUC remained strong at **0.9597**, confirming model stability.

- **Improvement through tuning**: Performance gains were observed after tuning of hyperparameters, validating the effectiveness of Optuna in our workflow.

In [231]:
# Save the Best Model
joblib.dump(best_lgbm_model, "models/best_lightgbm_model.pkl")

# Save Input Feature Columns Used For Training
input_cols = X_train_lgbm.columns.tolist()
joblib.dump(input_cols, "models/input_columns.pkl")

['models/input_columns.pkl']